# Db2 Go API Documentation
The following document summarizes the Go language support for Db2.

## Database APIs

### Database Connection 
* [Open a Connection to Database](#OpenApi)
* [Close a Connection](#CloseApi)

### Statement Execution
* [Prepare a Statement](#PrepareApi)
* [SQL Query](#QueryApi)
* [Execute a Prepared Statement](#ExecApi)
* [Query a Single Row](#QueryRowApi)
* [Return Answer Set Column Names](#ColumnsApi)
* [Read the Next Row](#NextApi)
* [Copy Row Values into Answer Set](#ScanApi)

### Transaction Management
* [Begin a Transaction](#BeginApi)
* [Commit a Transaction](#CommitApi)
* [Rollback a Transaction](#RollbackApi)

# API Conventions
All of the Db2 GO functions either return an error code, or a value, followed by an error code. A typical API call will have the following syntax:
```go
value, error = db.function(parameters,...)
```

There are three fields that are usually present in a call:
* value - An object or value being retrieved by the function
* error - The error status
* db - An identifier or object that the function is being performance against

For instance, if the function is being executed against a database, then the identifier `db` is expected to be a handle, or pointer to the database object that would have been created earlier in the code.

In all examples, the `error` field will contain `nil` if the function succeeded, otherwise it will be a non-null value. All functions should be followed by logic to trap any errors that occur with the function call:
```go
db, error := sql.Open("go_ibm_db",*connStr);
if error != nil {
    return error;
}
```

# Database Connections
Before you begin issuing any SQL commands, you must first establish a connection to a database. Then `Open` function is used to establish a connection to the database and it returns a database handle. The database handle will be used for subsequent calls to the database. In the event the connection was not successful, an error message and code will be returned to the application. 

When you have finished the application or transaction, you should close the connection handle with the `Close` function. If your program exits before issuing a `Close` command, there is always a possibility that the transaction will continue to be active in the Db2 database until a timeout occurs. To avoid situations where you exit the program without a `Close`, you can use the following Go syntax to fire the `Close` function at function exit.

```go
db, error = Open(...)
if (error == nil) {
    defer db.Close()
} else {
    Bad connection logic
}
```

You need to use the `if` block to check to see if a valid connection was returned. Otherwise, the `db.Close` function will cause an error on exit because there is no valid connection. Usually your application would have issued an error message about a bad connection in the body of the code.

The more traditional way of closing a database connection (Python, C, Java, etc...) would be to have the `Close` statement at the bottom of the code. This approach will also work but requires that any error situation will need to issue the `Close` statement, while the `defer` approach only requires one `Close` statement.

<a name="OpenApi"></a>
## Open a Connection to a Database 

Syntax:
```go
db, err = Open(drivername,ConnectionString)
```
* `drivername` - Always set to `go_ibm_db`
* `connectionString` - The connection string for your database

For distributed platforms, the connection string is typically defined as: 
```
DATABASE=dbname;HOSTNAME=hostname;PORT=port;PROTOCOL=TCPIP;UID=username;PWD=passwd
```

The parameters in this string follow the Db2 CLI/ODBC conventions.

Returns:

* `conn` - Connection handle used for subsequent calls
* `error` - Error details

Example:
```go
var connStr = flag.String("go_ibm_db", 
                          "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1")

db, err := sql.Open("go_ibm_db",*connStr);
```

<a name="CloseApi"></a>
## Close Database Connection

This function will close the specified database connection.

Syntax:
```go
db.Close()
```
* db - Current database handle

Example:
```go
defer db.Close()
```

As mentioned in the introduction to this section, you may want to add the `defer db.Close()` statement immediately after a successful connection so that you are guaranteed the connection will be closed upon exit from the application.

# Commit Scope
When a database like Db2 **commits** work, it means that the data has been hardened into the database. The changes are made to the underlying tables and you cannot **undo** the work. The default behavior for Db2 connections is **AUTOCOMMIT** which implies that every successful statement execution will have the work commited to the database. This behavior may be appropriate for some types of database calls (i.e. CREATE a table), but may not be correct when dealing with a financial transaction. A typical example would be to take money from one account and move it to another. If you commited the withdrawal (debit) on the first account and committed it, and the second part of the transaction (credit) failed, you would end up with inconsistent data and an unhappy customer.

To avoid these inconsistent situations, databases have the concept of commit scopes. You need to **begin** a transaction and then issue all of your INSERT, UPDATE, and DELETE statements as required. Once you have completed all of the steps, you will then issue a **COMMIT** statement to update all tables involved in the transaction. If there was an issue with the transaction, you could issue a **ROLLBACK** and remove all of the changes that were done and leave the tables in the same state as when you started the transaction. The other benefit of using transactions is that the database guarantees that your data will not change during the duration of your transaction. No other user will be able to update the data while you manipulating it.

A commit scope is always connected to a transaction handle, not the database. You are committing the work done by the transaction and so the `Commit` and `Rollback` refer to the transaction handle.

From a Go perspective, your code would look similar to the following:
```go
db, error = Open(...)

tx, error = db.Begin()

... do some work...

if everything is okay {
    tx.Commit()
} else {
    tx.Rollback()
}
```

Once you have committed or rolled back a transaction, you can continue to use the transaction handle for subsequent SQL calls.

<a name="BeginApi"></a> 
## Begin a SQL Transaction
When you begin a transaction, the program must issue either a `COMMIT` to complete the transaction, or a `ROLLBACK` to discard the changes.

Syntax:
```go
tx, error = db.Begin()
```
* `db` - The connection handle that was returned from the `open` connection call.

Returns:

* `tx` - Transaction handle that has a commit scope
* `error` - Error details

Example:
```go
tx, error := db.Begin()
```

<a name="CommitApi"></a> 
## Commit a Transaction
Commit a current transaction. All of the changes made to tables and database objects will be hardened (committed) to the database.

Syntax:
```go
error = tx.Commit()
```

* `tx` - The transaction handle that was generated earlier by a `Begin()` function.

Returns:

* `error` - Error details

Example:
```go
tx, error := db.Begin()
... SQL statements
error = bg.Commit()
if error != nil {
    return error
}
```

<a name="CommitApi"></a> 
## Rollback a Transaction
Rollback a current transaction. All of the changes made to tables and database objects will be discarded and the tables and database objects will revert back to the state they were in before you started the transaction.

Syntax:
```go
error = tx.Rollback()
```

* `tx` - The transaction handle that was generated earlier by a `Begin()` function.

Returns:

* `error` - Error details

Example:
```go
tx, error := db.Begin()
... SQL statements
error = bg.Rollback()
if error != nil {
    return error
}
```

# Single SQL Statement Execution
Once you have established a connection, there is a variety of functions that you can call to manipulate data in the database. The functions are:
* Query a Database
* Single Row Query
* Prepare a Statement
* Execute a Statement
* Columns
* Next Row
* Scan

## Statement Versus Database Execution
Many of these functions allow for execution against a database using a database handle, or execution with a statement handle. The difference depends on the type of SQL being executed, how often you are running it, and what commit scope you are using.



<a name="QueryApi"></a> 
##  Issue a Query Against a Database

Syntax:
```
rows, err = db.Query("sql")
```

* db - Current database handle
* sql - The SQL that is going to be executed

Returns:
* rows - Answer set 
* error - Error details

```go
rows,error := db.Query("select * from ak")
```

<a name="PrepareApi"></a>
## Prepare a SQL Statement

Use this command to prepare a SQL statement for subsequent execution.

Syntax:
`stmt, error = db.Prepare(sqlquery)`

* `db` - The connection handle that was returned from the `open` connection call.
* `sqlquery` - SQL string that you want to prepare

Returns:

* `stmt` - Statement object used for subsequent calls
* `error` - Error details


```go
stmt, error := db.Prepare("select * from ak")
```

Once you prepare a statement, you could use the `Query` function to execute it.

<a name="ExecApi"></a> 
## Execute a DDL Statement
Use this statement to execute SQL that does not return rows. This is typically used for DDL statements rather than SQL.

Syntax:
```
db.Exec(sqlquery)
```
* `db` - The connection handle that was returned from the `open` connection call.
* `sqlquery` - SQL string that you want to execute

Returns:

* `error` - Error details

Note the use of the `_` operator to indicate any results are to be ignored from this call.

Example:
```go
_,error = db.Exec("create table ghh(a int, b float, c double,  d char, e varchar(30))")
```

### <a name="QueryRowApi"></a> 9) .QueryRow(sqlquery)

QueryRow executes a query that is expected to return at most one row.
If there are more rows then it will scan first and discards the rest.
 
```go

func oper()  error {
    id := 123
    var username string
    err := db.QueryRow("SELECT name FROM ak WHERE id=?", id).Scan(&username)
    if err != nil {
        return err
    }

    fmt.Printf("Username is %s\n", username)
    return nil
}

```

### <a name="ColumnsApi"></a> 10) .Columns()

Returns the column names.

Returns error if the rows are closed.

```go

func oper() error {
    fmt.Println("connecting to databse");
    db, err := sql.Open("go-ibm_db",*connStr);
    if err != nil {
        return err;
    }

    defer db.Close()

    st, err := db.Prepare("select * from ak")
    if err !=nil {
    return err
    }
    
    rows,err :=st.Query()
    if err != nil {
        return err
    }
    
    defer rows.Close()
    name11 := make([]string,1)
    name11, err = rows.Columns()
    fmt.Printf("%v",name11);
    return nil
}
```

### <a name="NextApi"></a> 11) .Next()

Prepares the next result row for reading with the scan api.

```go

func oper() error {
    fmt.Println("connecting to database");
    db, err:=sql.Open("go-ibm_db",*connStr);
    if err != nil {
        return err;
    }

    defer db.Close()
    rows,err := db.Query()
    if err != nil {
        return err
    }
    
    defer rows.Close()
    for rows.Next() {
        var t string
        var x string
        err = rows.Scan(&t, &x)
        if err != nil {
            return err
        }

        fmt.Printf("%v %v\n",t,x)
    }
    
    return nil
}
```

### <a name="ScanApi"></a> 12) .Scan(options)

copies the columns in the current row into the values pointed.

```go

func oper() error {
    fmt.Println("connecting to database");
    db, err := sql.Open("go-ibm_db", *connStr);
    if err != nil {
        return err;
    }

    defer db.Close()

    rows,err := db.Query()
    if err != nil {
        return err
    }

    defer rows.Close()
    for rows.Next() {
        var t string
        var x string
        err = rows.Scan(&t, &x)
        if err != nil {
            return err
        }

        fmt.Printf("%v %v\n",t,x)
    }
    return nil
}
```

## Summary
In summary, you've learned how to start, create, update and edit Jupyter notebooks. Jupyter notebooks are used extensively by the data science community, but it is finding its way into many other areas as well. If you are interested in what other applications use Jupyter notebooks, take a look at the list maintained on this web site: https://github.com/jupyter/jupyter/wiki/A-gallery-of-interesting-Jupyter-Notebooks.

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]